# Retrieve NetCDF and model gridded climate time-series for a watershed

### Case study:  the Sauk-Suiattle Watershed
<img src="http://www.sauk-suiattle.com/images/Elliott.jpg" 
style="float:right;width:150px;padding:20px">

### Use this Jupyter Notebook to:
    1. HydroShare setup and preparation
    2. Re-establish the paths to the mapping file
    3. Compute daily, monthly, and annual temperature and precipitation statistics
    4. Visualize precipitation results relative to the forcing data
    5. Visualize the time-series trends
    6. Save results back into HydroShare

<br/><br/><br/>
<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png"
style="float:right;width:150px;padding:20px">

<br/><br/>
#### This data is compiled to digitally observe the watersheds, powered by HydroShare. <br/>Provided by the Watershed Dynamics Group, Dept. of Civil and Environmental Engineering, University of Washington

## 1.  Prepare HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

In [34]:
# silencing warning
import warnings
warnings.filterwarnings("ignore")

# data processing
import os
import pandas as pd
import numpy as np
import wget
import urllib
import dask as da
import xarray as xray
import sys

# data migration library
#import ogh
from utilities import hydroshare
import ogh_xarray_landlab as oxl

# plotting and shape libraries
import matplotlib.pyplot as plt
%matplotlib inline

# supplementary libraries
from dask.diagnostics import ProgressBar
from multiprocessing.pool import ThreadPool

Establish a secure connection with HydroShare by instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving work back to HydroShare. 

In [2]:
notebookdir = os.getcwd()

hs=hydroshare.hydroshare()
homedir = hs.getContentPath(os.environ["HS_RES_ID"])
os.chdir(homedir)

Adding the following system variables:
   HS_USR_NAME = jphuong
   HS_RES_ID = 87dc5742cf164126a11ff45c3307fd9d
   HS_RES_TYPE = compositeresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => jphuong

The hs_utils library requires a secure connection to your HydroShare account.
Enter the HydroShare password for user 'jphuong': ········
Successfully established a connection with HydroShare


If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## 2. Get list of gridded climate points for the watershed

For visualization purposes, we will also remap the study site shapefile, which is stored in HydroShare at the following url: https://www.hydroshare.org/resource/c532e0578e974201a0bc40a37ef2d284/. Since the shapefile was previously migrated, we can select 'N' for no overwriting.

In the usecase1 notebook, the treatgeoself function identified the gridded cell centroid coordinates that overlap with our study site. These coordinates were documented within the mapping file, which will be remapped here. In the usecase2 notebook, the downloaded files were cataloged within the mapping file, so we will use the mappingfileSummary function to characterize the files available for Sauk-Suiattle for each gridded data product.

In [3]:
"""
1/16-degree Gridded cell centroids
"""
# List of available data
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']


"""
Sauk
"""
# Watershed extent
hs.getResourceFromHydroShare('c532e0578e974201a0bc40a37ef2d284')
sauk = hs.content['wbdhub12_17110006_WGS84_Basin.shp']

# reproject the shapefile into WGS84
ogh.reprojShapefile(sourcepath=sauk)

This resource already exists in your userspace.
ef2d82bf960144b4bfb1bae6242bcc7f/
|-- ef2d82bf960144b4bfb1bae6242bcc7f/
|   |-- bagit.txt
|   |-- manifest-md5.txt
|   |-- readme.txt
|   |-- tagmanifest-md5.txt
|   |-- data/
|   |   |-- resourcemap.xml
|   |   |-- resourcemetadata.xml
|   |   |-- contents/
|   |   |   |-- NAmer_dem_list.cpg
|   |   |   |-- NAmer_dem_list.dbf
|   |   |   |-- NAmer_dem_list.prj
|   |   |   |-- NAmer_dem_list.sbn
|   |   |   |-- NAmer_dem_list.sbx
|   |   |   |-- NAmer_dem_list.shp
|   |   |   |-- NAmer_dem_list.shx

Do you want to overwrite these data [Y/n]? n


This resource already exists in your userspace.
c532e0578e974201a0bc40a37ef2d284/
|-- c532e0578e974201a0bc40a37ef2d284/
|   |-- bagit.txt
|   |-- manifest-md5.txt
|   |-- readme.txt
|   |-- tagmanifest-md5.txt
|   |-- data/
|   |   |-- resourcemap.xml
|   |   |-- resourcemetadata.xml
|   |   |-- contents/
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.cpg
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.dbf
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.prj
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.sbn
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.sbx
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.shp
|   |   |   |-- wbdhub12_17110006_WGS84_Basin.shx

Do you want to overwrite these data [Y/n]? n


### Summarize the file availability from each watershed mapping file

In [ ]:
#mappingfile1 = os.path.join(homedir, 'Sauk_mappingfile.csv')

In [ ]:
%%time

# map the mappingfiles from usecase1
mappingfile1=ogh.treatgeoself(shapefile=sauk, NAmer=NAmer, buffer_distance=0.06,
                              mappingfile=os.path.join(homedir,'Sauk_mappingfile.csv'))

## 3.  Compare Hydrometeorology 

This section performs computations and generates plots of the Livneh 2013 and Salathe 2014 mean temperature and mean total monthly precipitation in order to compare them with each other. The generated plots are automatically downloaded and saved as .png files within the "homedir" directory.

Let's compare the Livneh 2013 and Salathe 2014 using the period of overlapping history.

In [7]:
help(oxl.get_netcdfmap_hourlywrf_PNNL2018)

Help on function get_netcdfmap_hourlywrf_PNNL2018 in module ogh_xarray_landlab:

get_netcdfmap_hourlywrf_PNNL2018(spatialbounds, homedir, subdir='PNNL2018/Hourly_WRF_1981_2015/noBC', start_date='1970-01-01', end_date='1970-02-28', nworkers=20)
    get hourly WRF data from a 2018 PNNL WRF run using xarray on netcdf files



## NetCDF retrieval and clipping to a spatial extent

The function get_x_dailywrf_salathe2014 retrieves and clips NetCDF files archived within the UW Rocinante NNRP repository. This archive contains daily data from January 1970 through December 1979 (10 years). Each netcdf file is comprised of meteorologic and VIC hydrologic outputs for a calendar month. The expected number of files would be 360 files (12 months for 30 years). 

In the code chunk below, 40 parallel workers will be initialized to distribute file retrieval and spatial clipping tasks. For each worker, they will wget the requested file, clip the netcdf file to gridded cell centroids within the the provided bounding box, then return the location of the spatially clipped output files.

Provide the home and subdirectory where the cropped NetCDF files will be stored. Also provide the spatial bounds (in WGS84) to crop the NetCDF files upon download. Finally, provide the number of workers to carry out the download tasks, and the start and end date of the files of interest.

In [64]:
# get the coordinate map
domain = 'http://cses.washington.edu'
subdomain = 'rocinante/WRF/PNNL_NARR_6km'
netcdfmap = 'data_LatLonGht.nc'

# retrieve the map
if not os.path.exists(netcdfmap):
    wget.download('{0}/{1}/{2}'.format(domain, subdomain, netcdfmap))

# open the map
pnnlxy=xray.open_dataset(netcdfmap)
pnnlxy

<xarray.Dataset>
Dimensions:  (south_north: 88, west_east: 75)
Dimensions without coordinates: south_north, west_east
Data variables:
    LAT      (south_north, west_east) float32 ...
    LON      (south_north, west_east) float32 ...
    Z        (south_north, west_east) float32 ...

## Download a few sample NetCDF data files

In [31]:
start_date='1981-01-01'
end_date='1981-01-31'

dr = pd.date_range(start=start_date, end=end_date, freq='D')
filelist = oxl.compile_x_wrfpnnl2018_raw_locations(dr,
                                                   domain='http://cses.washington.edu',
                                                   subdomain='rocinante/WRF/PNNL_NARR_6km')
filelist

['http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-01.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-02.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-03.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-04.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-05.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-06.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-07.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-08.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-09.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-10.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-11.nc',
 'http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-12.nc',
 'ht

In [33]:
def compile_x_wrfpnnl2018_raw_locations(time_increments,
                                        domain='http://cses.washington.edu',
                                        subdomain='rocinante/WRF/PNNL_NARR_6km'):
    """
    Compile a list of file URLs for PNNL 2018 raw WRF data
    time_increments: (list) a list of dates that identify each netcdf file
    """
    locations=[]

    for ind, ymd in enumerate(time_increments):
        subfolder = '{0}'.format(ymd.strftime('%Y'))
        basename='data.{0}.nc'.format(ymd.strftime('%Y-%m-%d'))
        url='{0}/{1}/{2}/{3}'.format(domain, subdomain, subfolder, basename)
        locations.append(url)
    return(locations)


def ensure_dir(f):
    """
    Check if the folder directory exists, else create it, then set it as the working directory
    
    f: (dir) the directory to create and/or set as working directory
    """
    if not os.path.exists(f):
        os.makedirs(f)
    os.chdir(f)
    
    
def wget_download_one(fileurl):
    """
    Download a file from an http domain
    
    fileurl: (url) a url to request
    """
    # check and download each location point, if it doesn't already exist in the download directory
    basename=os.path.basename(fileurl)
    
    # if it exists, remove for new download (overwrite mode)
    if os.path.isfile(basename):
        os.remove(basename)
        
    try:
        ping = urllib.request.urlopen(fileurl)
        if ping.getcode()!=404:
            wget.download(fileurl)
            print('downloaded: ' + basename)
    except:
        print('File does not exist at this URL: ' + basename)
        
        
def wget_download_p(listofinterest, nworkers=20):
    """
    Download files from an http domain in parallel
    
    listofinterest: (list) a list of urls to request
    nworkers: (int) the number of processors to distribute tasks; default is 20
    """
    # initialize parallel workers
    #da.set_options(pool=ThreadPool(nworkers))
    #ProgressBar().register()
    #pool = dask.delayed(wget_download_one)(each for each in listofinterest)
    #dask.compute(pool)
    pool = Pool(int(nworkers))
    pool.map(wget_download_one, listofinterest)
    pool.close()
    pool.terminate()
    
    
def wget_netcdfmap_spSubset(url,
                            datetime,
                            spatialbounds,
                            netcdfmap=netcdfmap,
                            nworkers=20,
                            time_resolution='H',
                            time_steps=24,
                            file_prefix='sp_',
                            rename_timelatlong_names={'LAT':'LAT','LON':'LON'},
                            replace_file=True):
    
    # retrieve the file
    #ogh.wget_download_one(url)
    wget_download_one(url)
    
    # open the file
    ds = xray.open_dataset(os.path.basename(url), engine='netcdf4')

    #merge pnnl lat long data with climate data
    ds_new = xr.merge([ds, netcdfmap], compat='no_conflicts')

    #convert variables LAT, LON and Z to coordinates
    ds_c = ds_new.set_coords({'LAT','LON','Z'}, inplace=False)

    #create series of dates to add to dataset
    time_inc = pd.date_range(start=ymd, periods=time_steps, freq=time_resolution)

    #add coordinates using series of dates
    ds_c.update({'time': ('time', time_inc)})
    
    # rename latlong if they are not LAT and LON, respectively
    if not isinstance(rename_timelatlong_names, type(None)):
        ds_c = ds_c.rename(rename_timelatlong_names)

    # slice by the bounding box
    spSubset = ds_c.sel(LON=slice(spatialbounds['minx'], spatialbounds['maxx']),
                        LAT=slice(spatialbounds['miny'], spatialbounds['maxy']))

    # print the spatial subset
    spSubset.to_netcdf(file_prefix+os.path.basename(url))
    
    # remove the parent
    ds.close()
    os.remove(os.path.basename(url))
    return(os.path.join(os.getcwd(), file_prefix+os.path.basename(url)))

In [38]:
print(filelist[0])
ping = urllib.request.urlopen(filelist[0])
if ping.getcode()!=404:
    wget.download(filelist[0])

http://cses.washington.edu/rocinante/WRF/PNNL_NARR_6km/1981/data.1981-01-01.nc


In [44]:
def get_netcdfmap_hourlywrf_PNNL2018(spatialbounds,
                                     homedir,
                                     subdir='PNNL2018/Hourly_WRF_1981_2015/noBC',
                                     start_date='1970-01-01',
                                     end_date='1970-02-28',
                                     nworkers=20):
    """
    get hourly WRF data from a 2018 PNNL WRF run using xarray on netcdf files
    """
    # retrieve the netcdf map
    domain='http://cses.washington.edu'
    subdomain='rocinante/WRF/PNNL_NARR_6km'
    netcdfmap = 'data_LatLonGht.nc'
    if not os.path.exists(netcdfmap):
        wget.download('{0}/{1}/{2}'.format(domain, subdomain, netcdfmap))
    pnnlxy=xray.open_dataset(netcdfmap)
    
    # check and generate data directory
    filedir=os.path.join(homedir, subdir)
    #ogh.ensure_dir(filedir)
    ensure_dir(filedir)

    # initialize parallel workers
    da.set_options(pool=ThreadPool(nworkers))
    ProgressBar().register()

    # map to the data files
    locations=[]
    dr = pd.date_range(start=start_date, end=end_date, freq='D')
    locations = compile_x_wrfpnnl2018_raw_locations(time_increments=dr,
                                                    domain=domain,
                                                    subdomain=domain)
    
    # retrieve and subset each netcdf datafile
    pnnl_files=[]
    for eachurl, ymd in zip(locations, dr):
        pnnl_files.append(da.delayed(wget_netcdfmap_spSubset)(url=eachurl,
                                                              datetime=ymd,
                                                              spatialbounds=spatialbounds,
                                                              netcdfmap=netcdfmap,
                                                              nworkers=nworkers,
                                                              time_resolution='H',
                                                              file_prefix='sp_',
                                                              rename_timelatlong_names={'LAT':'LAT','LON':'LON'},
                                                              replace_file=True))
    # execute retrieval and spSubsetting
    #pnnl_spSubset = da.compute(pnnl_files)[0]
    #return(pnnl_spSubset)
    return(pnnl_files)


def compareonvar(map_df, colvar='all'):
    """
    subsetting a dataframe based on some columns of interest
    
    map_df: (dataframe) the dataframe of the mappingfile table
    colvar: (str or list) the column(s) to use for subsetting; 'None' returns an outerjoin, 'all' returns an innerjoin
    """
    # apply row-wise inclusion based on a subset of columns
    if isinstance(colvar, type(None)):
        return(map_df)
    
    if colvar is 'all':
        # compare on all columns except the station info
        return(map_df.dropna())
    else:
        # compare on only the listed columns
        return(map_df.dropna(subset=colvar))

    
def mappingfileToDF(mappingfile, colvar='all', summary=True):
    """
    read in a dataframe and subset based on columns of interest
    
    mappingfile: (dir) the file path to the mappingfile, which contains LAT, LONG_, and ELEV coordinates of interest
    colvar: (str or list) the column(s) to use for subsetting; 'None' returns an outerjoin, 'all' returns an innerjoin
    """
    # Read in the mappingfile as a data frame
    map_df = pd.read_csv(mappingfile)
    
    # select rows (datafiles) based on the colvar(s) chosen, default is 
    map_df = compareonvar(map_df=map_df, colvar=colvar)
    
    if summary:
        # compile summaries
        print('Number of gridded data files:'+ str(len(map_df)))
        print('Minimum elevation: ' + str(np.min(map_df.ELEV))+ 'm')
        print('Mean elevation: '+ str(np.mean(map_df.ELEV))+ 'm')
        print('Maximum elevation: '+ str(np.max(map_df.ELEV))+ 'm')
        
    return(map_df, len(map_df))


In [45]:
#maptable, nstations = ogh.mappingfileToDF(mappingfile1)
maptable, nstations = mappingfileToDF(mappingfile1)
spatialbounds = {'minx':maptable.LONG_.min(), 'maxx':maptable.LONG_.max(),
                 'miny':maptable.LAT.min(), 'maxy':maptable.LAT.max()}

outputfiles = get_netcdfmap_hourlywrf_PNNL2018(homedir=homedir,
                                               subdir='PNNL2018/Hourly_WRF_1970_1970/raw_netcdf',
                                               spatialbounds=spatialbounds,
                                               nworkers=5,
                                               start_date=start_date, 
                                               end_date=end_date)

Number of gridded data files:99
Minimum elevation: 164.0m
Mean elevation: 1151.040404040404m
Maximum elevation: 2216.0m


In [62]:
#netcdfmap
os.path.join(os.getcwd(),netcdfmap)

'/home/jovyan/work/notebooks/data/87dc5742cf164126a11ff45c3307fd9d/87dc5742cf164126a11ff45c3307fd9d/data/contents/PNNL2018/Hourly_WRF_1970_1970/raw_netcdf/data_LatLonGht.nc'

In [68]:
#perform = da.compute(outputfiles[0])[0]

spatialbounds
pnnlxy=xray.open_dataset(os.path.join(os.getcwd(),netcdfmap), engine='netcdf4')

nworkers=20
time_resolution='H'
time_steps=24
file_prefix='sp_'
rename_timelatlong_names={'LAT':'LAT','LON':'LON'}
replace_file=True

for url, ymd in zip(filelist[:3], dr[:3]):
    
    # retrieve the file
    #ogh.wget_download_one(url)
    wget_download_one(url)
    
    # open the file
    ds = xray.open_dataset(os.path.basename(url), engine='netcdf4')

    #merge pnnl lat long data with climate data
    ds_new = xray.merge([ds, pnnlxy], compat='no_conflicts')

    #convert variables LAT, LON and Z to coordinates
    ds_c = ds_new.set_coords({'LAT','LON','Z'}, inplace=False)

    #create series of dates to add to dataset
    time_inc = pd.date_range(start=ymd, periods=time_steps, freq=time_resolution)

    #add coordinates using series of dates
    ds_c.update({'time': ('time', time_inc)})
    
    # rename latlong if they are not LAT and LON, respectively
    if not isinstance(rename_timelatlong_names, type(None)):
        ds_c = ds_c.rename(rename_timelatlong_names)

    print(ds_c)
    # slice by the bounding box
    spSubset = ds_c.sel(LON=slice(spatialbounds['minx'], spatialbounds['maxx']),
                        LAT=slice(spatialbounds['miny'], spatialbounds['maxy']))

    # print the spatial subset
    spSubset.to_netcdf(file_prefix+os.path.basename(url))

downloaded: data.1981-01-01.nc
<xarray.Dataset>
Dimensions:      (south_north: 88, time: 24, west_east: 75)
Coordinates:
    LAT          (south_north, west_east) float32 ...
    LON          (south_north, west_east) float32 ...
    Z            (south_north, west_east) float32 ...
  * time         (time) datetime64[ns] 1981-01-01 ... 1981-01-01T23:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    T2           (time, south_north, west_east) float32 ...
    Q2           (time, south_north, west_east) float32 ...
    PSFC         (time, south_north, west_east) float32 ...
    GLW          (time, south_north, west_east) float32 ...
    SWDOWN       (time, south_north, west_east) float32 ...
    U10          (time, south_north, west_east) float32 ...
    V10          (time, south_north, west_east) float32 ...
    PREC_ACC_NC  (time, south_north, west_east) float32 ...
    SNOW_ACC_NC  (time, south_north, west_east) float32 ...


ValueError: dimensions or multi-index levels ['LON', 'LAT'] do not exist

In [99]:
ds_c.sel(time=slice('1981-01-01T018:00:00.000000000', '1981-01-01T21:00:00.000000000'))

<xarray.Dataset>
Dimensions:      (south_north: 88, time: 4, west_east: 75)
Coordinates:
    LAT          (south_north, west_east) float32 45.0204 45.028 ... 50.1414
    LON          (south_north, west_east) float32 ...
    Z            (south_north, west_east) float32 0.0 0.0 ... 1382.39 1516.88
  * time         (time) datetime64[ns] 1981-01-01T18:00:00 ... 1981-01-01T21:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    T2           (time, south_north, west_east) float32 ...
    Q2           (time, south_north, west_east) float32 ...
    PSFC         (time, south_north, west_east) float32 ...
    GLW          (time, south_north, west_east) float32 ...
    SWDOWN       (time, south_north, west_east) float32 ...
    U10          (time, south_north, west_east) float32 ...
    V10          (time, south_north, west_east) float32 ...
    PREC_ACC_NC  (time, south_north, west_east) float32 ...
    SNOW_ACC_NC  (time, south_north, west_east) float32 ...

In [100]:
ds_c.sel(LON=slice(spatialbounds['minx'], spatialbounds['maxx']))

ValueError: dimensions or multi-index levels ['LON'] do not exist

In [132]:
ds_c.T2.south_north

<xarray.DataArray 'south_north' (south_north: 88)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87])
Dimensions without coordinates: south_north

In [123]:
LATs=np.array([])
for ind, eachline in enumerate(ds_c.LAT.data):
    LATs=np.concatenate([LATs,eachline])

LONs=np.array([])
for ind, eachline in enumerate(ds_c.LON.data):
    LONs=np.concatenate([LONs,eachline])

#add coordinates using series of dates
ds_c.update({'LATs': ('LATs', LATs)})
ds_c.update({'LONs': ('LONs', LONs)})

#convert variables LAT, LON and Z to coordinates
ds_c = ds_new.set_coords({'LAT','LON','Z'}, inplace=False)

<xarray.Dataset>
Dimensions:      (LATs: 6600, LONs: 6600, south_north: 88, time: 24, west_east: 75)
Coordinates:
    LAT          (south_north, west_east) float32 45.0204 45.028 ... 50.1414
    LON          (south_north, west_east) float32 -124.985 -124.909 ... -119.851
    Z            (south_north, west_east) float32 0.0 0.0 ... 1382.39 1516.88
  * time         (time) datetime64[ns] 1981-01-01 ... 1981-01-01T23:00:00
  * LATs         (LATs) float64 45.02 45.03 45.04 45.04 ... 50.13 50.14 50.14
  * LONs         (LONs) float64 -125.0 -124.9 -124.8 ... -120.0 -119.9 -119.9
Dimensions without coordinates: south_north, west_east
Data variables:
    T2           (time, south_north, west_east) float32 ...
    Q2           (time, south_north, west_east) float32 ...
    PSFC         (time, south_north, west_east) float32 ...
    GLW          (time, south_north, west_east) float32 ...
    SWDOWN       (time, south_north, west_east) float32 ...
    U10          (time, south_north, west_east) f

In [ ]:
#add coordinates using series of dates
ds_c.update({'LATs': ('LATs', LATs)})
ds_c.update({'LONs': ('LONs', LONs)})

In [140]:
ds_c.T2

<xarray.DataArray 'T2' (time: 24, south_north: 88, west_east: 75)>
[158400 values with dtype=float32]
Coordinates:
    LAT      (south_north, west_east) float32 45.0204 45.028 ... 50.137 50.1414
    LON      (south_north, west_east) float32 -124.985 -124.909 ... -119.851
    Z        (south_north, west_east) float32 0.0 0.0 0.0 ... 1382.39 1516.88
  * time     (time) datetime64[ns] 1981-01-01 ... 1981-01-01T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    units:        K
    description:  Air Temperature at 2m 

In [130]:
#ds_c.T2.sel(south_north=slice(spatialbounds['minx'], spatialbounds['maxx']))
ds_c.T2.sel(time=slice('1981-01-01T018:00:00.000000000', '1981-01-01T21:00:00.000000000'))

<xarray.DataArray 'T2' (time: 4, south_north: 88, west_east: 75)>
array([[[ 283.06781 ,  282.995697, ...,  279.703918,  279.567993],
        [ 283.088135,  282.92807 , ...,  280.379639,  280.331268],
        ..., 
        [ 282.743896,  281.649963, ...,  275.711853,  274.920471],
        [ 283.677765,  281.619598, ...,  276.210236,  275.094177]],

       [[ 283.027313,  283.578583, ...,  280.052124,  279.990326],
        [ 283.065063,  283.437988, ...,  280.683777,  280.657776],
        ..., 
        [ 283.939545,  282.839264, ...,  276.178528,  275.262543],
        [ 285.08432 ,  282.696991, ...,  276.938965,  275.865784]],

       [[ 284.048615,  284.502838, ...,  280.264496,  280.316132],
        [ 284.031311,  284.436493, ...,  280.890442,  280.883484],
        ..., 
        [ 284.705597,  283.593445, ...,  276.522308,  275.484924],
        [ 286.000061,  283.3479  , ...,  277.37558 ,  276.232941]],

       [[ 284.727783,  284.86673 , ...,  280.600555,  280.891388],
        [ 284.6

In [125]:
spSubset = ds_c.sel(south_north=slice(spatialbounds['minx'], spatialbounds['maxx']),
                    LATs=slice(spatialbounds['miny'], spatialbounds['maxy']))

spSubset

KeyError: -121.71875

In [49]:
os.listdir('/home/jovyan/work/notebooks/data/87dc5742cf164126a11ff45c3307fd9d/87dc5742cf164126a11ff45c3307fd9d/data/contents/PNNL2018/Hourly_WRF_1970_1970/raw_netcdf/')

['data_LatLonGht.nc']

In [48]:
#tmp = xray.open_dataset('/home/jovyan/work/notebooks/data/87dc5742cf164126a11ff45c3307fd9d/87dc5742cf164126a11ff45c3307fd9d/data/contents/PNNL2018/Hourly_WRF_1970_1970/raw_netcdf/data.1981-01-01.nc')
tmp = xray.open_dataset('data.1981-01-01.nc')
tmp

FileNotFoundError: [Errno 2] No such file or directory: b'/home/jovyan/work/notebooks/data/87dc5742cf164126a11ff45c3307fd9d/87dc5742cf164126a11ff45c3307fd9d/data/contents/PNNL2018/Hourly_WRF_1970_1970/raw_netcdf/data.1981-01-01.nc'

### Convert collection of NetCDF files into a collection of ASCII files

Provide the home and subdirectory where the ASCII files will be stored, the source_directory of netCDF files, and the mapping file to which the resulting ASCII files will be cataloged. Also, provide the Pandas Datetime code for the frequency of the time steps. Finally, provide the catalog label that will be used for the mapping file catalog and the metadata file label.

In [ ]:
%%time
# convert the netCDF files into daily ascii time-series files for each gridded location
outfilelist = oxl.netcdf_to_ascii(homedir=homedir, 
                                  subdir='livneh2013/Daily_MET_1970_1970/raw_ascii', 
                                  source_directory=os.path.join(homedir, 'livneh2013/Daily_MET_1970_1970/raw_netcdf'),
                                  mappingfile=mappingfile1,
                                  temporal_resolution='D',
                                  meta_file=meta_file,
                                  catalog_label='sp_dailymet_livneh_1970_1970')

In [ ]:
t1 = ogh.mappingfileSummary(listofmappingfiles = [mappingfile1], 
                            listofwatershednames = ['Sauk-Suiattle river'],
                            meta_file=meta_file)

t1

In [ ]:
# Save the metadata
ogh.saveDictOfDf(dictionaryObject=meta_file, outfilepath='test.json')

### Create a dictionary of climate variables for the long-term mean (ltm).
#### INPUT: gridded meteorology ASCII files located from the Sauk-Suiattle Mapping file. The inputs to gridclim_dict() include the folder location and name of the hydrometeorology data, the file start and end, the analysis start and end, and the elevation band to be included in the analsyis (max and min elevation). <br/>OUTPUT: dictionary of dataframes where rows are temporal summaries and columns are spatial summaries

In [ ]:
meta_file['sp_dailymet_livneh_1970_1970']['variable_list']

In [ ]:
%%time

ltm = ogh.gridclim_dict(mappingfile=mappingfile1,
                        metadata=meta_file,
                        dataset='sp_dailymet_livneh_1970_1970',
                        variable_list=['Prec','Tmax','Tmin'])

sorted(ltm.keys())

### Compute the total monthly and yearly precipitation, as well as the mean values across time and across stations
#### INPUT: daily precipitation for each station from the long-term mean dictionary (ltm) <br/>OUTPUT: Append the computed dataframes and values into the ltm dictionary

In [ ]:
# extract metadata
dr = meta_file['sp_dailymet_livneh_1970_1970']

# compute sums and mean monthly an yearly sums
ltm = ogh.aggregate_space_time_sum(df_dict=ltm,
                                   suffix='Prec_sp_dailymet_livneh_1970_1970',
                                   start_date=dr['start_date'],
                                   end_date=dr['end_date'])

In [ ]:
# print the name of the analytical dataframes and values within ltm
sorted(ltm.keys())

In [ ]:
# initialize list of outputs
files=[]

# create the destination path for the dictionary of dataframes
ltm_sauk=os.path.join(homedir, 'ltm_1970_1970_sauk.json')
ogh.saveDictOfDf(dictionaryObject=ltm, outfilepath=ltm_sauk)
files.append(ltm_sauk)

# append the mapping file for Sauk-Suiattle gridded cell centroids
files.append(mappingfile1)

### Visualize the "average monthly total precipitations"

#### INPUT: dataframe with each month as a row and each station as a column. <br/>OUTPUT: A png file that represents the distribution across stations (in Wateryear order)

In [ ]:
# # two lowest elevation locations
lowE_ref = ogh.findCentroidCode(mappingfile=mappingfile1, colvar='ELEV', colvalue=164)

# one highest elevation location
highE_ref = ogh.findCentroidCode(mappingfile=mappingfile1, colvar='ELEV', colvalue=2216)

# combine references together
reference_lines = highE_ref + lowE_ref
reference_lines


In [ ]:
ogh.renderValueInBoxplot(ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'],
                         outfilepath='totalMonthlyRainfall.png', 
                         plottitle='Total monthly rainfall',
                         time_steps='month',
                         wateryear=True,
                         reference_lines=reference_lines,
                         ref_legend=True,
                         value_name='Total daily precipitation (mm)',
                         cmap='seismic_r',
                         figsize=(6,6))

In [ ]:
ogh.renderValuesInPoints(ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'], 
                         vardf_dateindex=12, 
                         shapefile=sauk, 
                         cmap='seismic_r',
                         outfilepath='test.png', 
                         plottitle='December total rainfall',
                         colorbar_label='Total monthly rainfall (mm)', 
                         figsize=(1.5,1.5))

In [ ]:
minx2, miny2, maxx2, maxy2 = oxl.calculateUTMbounds(mappingfile=mappingfile1,
                                                    mappingfile_crs={'init':'epsg:4326'},
                                                    spatial_resolution=0.06250)

print(minx2, miny2, maxx2, maxy2)

# generate a raster

In [ ]:
help(oxl.rasterDimensions)

In [ ]:
# generate a raster
raster, row_list, col_list = oxl.rasterDimensions(minx=minx2, miny=miny2, maxx=maxx2, maxy=maxy2, dx=1000, dy=1000)
raster.shape

# Higher resolution children of gridded cells 
### get data from Lower resolution parent grid cells to the children

In [ ]:
help(oxl.mappingfileToRaster)

In [ ]:
%%time

# landlab raster node crossmap to gridded cell id
nodeXmap, raster, m = oxl.mappingfileToRaster(mappingfile=mappingfile1, spatial_resolution=0.06250, 
                                           minx=minx2, miny=miny2, maxx=maxx2, maxy=maxy2, dx=1000, dy=1000)

In [ ]:
# print the raster dimensions
raster.shape

In [ ]:
%%time
nodeXmap.plot(column='ELEV', figsize=(10,10), legend=True)

In [ ]:
# generate vector array of December monthly precipitation
prec_vector = ogh.rasterVector(vardf=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'],
                          vardf_dateindex=12,
                          crossmap=nodeXmap,
                          nodata=-9999)

# close-off areas without data
raster.status_at_node[prec_vector==-9999] = CLOSED_BOUNDARY

fig =plt.figure(figsize=(10,10))
imshow_grid(raster, 
            prec_vector,
            var_name='Daily precipitation',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Prec'].attrs['units'],
            color_for_closed='black', 
            cmap='seismic_r')

In [ ]:
tmax_vector = ogh.rasterVector(vardf=ltm['meanbymonth_Tmax_sp_dailymet_livneh_1970_1970'],
                          vardf_dateindex=12,
                          crossmap=nodeXmap,
                          nodata=-9999)

fig = plt.figure(figsize=(10,10))
imshow_grid(raster, 
            tmax_vector,
            var_name='Daily maximum temperature',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Tmax'].attrs['units'],
            color_for_closed='black', symmetric_cbar=False, cmap='magma')

In [ ]:
tmin_vector = ogh.rasterVector(vardf=ltm['meanbymonth_Tmin_sp_dailymet_livneh_1970_1970'],
                          vardf_dateindex=12,
                          crossmap=nodeXmap,
                          nodata=-9999)

fig = plt.figure(figsize=(10,10))
imshow_grid(raster, 
            tmin_vector,
            var_name='Daily minimum temperature',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Tmin'].attrs['units'],
            color_for_closed='black', symmetric_cbar=False, cmap='magma')

In [ ]:
# convert a raster vector back to geospatial presentation
t4, t5 = rasterVectorToWGS(prec_vector, nodeXmap=nodeXmap, UTM_transformer=m)

In [ ]:
t4.plot(column='value', figsize=(10,10), legend=True)

In [ ]:
# this is one decade
inputvectors = {'precip_met': np.tile(ltm['meandaily_Prec_sp_dailymet_livneh_1970_1970'], 15000),
                'Tmax_met': np.tile(ltm['meandaily_Tmax_sp_dailymet_livneh_1970_1970'], 15000),
                'Tmin_met': np.tile(ltm['meandaily_Tmin_sp_dailymet_livneh_1970_1970'], 15000)}

In [ ]:
%%time
(VegType_low, yrs_low, debug_low) = run_ecohydrology_model(raster,
                                                           input_data=inputvectors,
                                                           input_file=InputFile,
                                                           synthetic_storms=False,
                                                           number_of_storms=100000,
                                                           pet_method='PriestleyTaylor')

In [ ]:
plot_results(raster, VegType_low, yrs_low, yr_step=yrs_low-1)
plt.show()
plt.savefig('grid_low.png')

### Visualize the "average monthly total precipitation"

## 5. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

### Total files and image to migrate

In [ ]:
len(files)

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Computed spatial-temporal summaries of two gridded data product data sets for Sauk-Suiattle'
abstract = 'This resource contains the computed summaries for the Meteorology data from Livneh et al. 2013 and the WRF data from Salathe et al. 2014.'
keywords = ['Sauk-Suiattle', 'Livneh 2013', 'Salathe 2014','climate','hydromet','watershed', 'visualizations and summaries'] 
rtype = 'genericresource'

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)